In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os

In [2]:
BATCH_SIZE = 64
#EPISODE = 500000
EPISODE = 300000
TEST_EPISODE = 1000
LEARNING_RATE =5e-4
Nom = 1e-5
Weight_Deacy =1e-5
GPU = 0
Margin = 0.3

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'CUB1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_2 import AttributeNetwork
from my_net_2 import TripletNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(312,1200,1200,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
triplet_network = TripletNetwork(attribute_network)  #metric learning   
triplet_network.cuda(GPU)   
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
#triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE)
#
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE, 
                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=100000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*1024

        #print(batch_features_ext)

        sample_features = attribute_network(Variable(sample_attributes).cuda(GPU).float())
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
print("training...")
last_accuracy = 0.0
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()[i]!=train_id)[0], replace=True)
        batch_features_negative[i] = x[pick]    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(batch_attributes, batch_features, batch_features_negative)
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet + Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss.data)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy(test_features, test_label, np.arange(200), attributes)
        gzsl_seen_accuracy = compute_accuracy(test_seen_features, test_seen_label, np.arange(200), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)

training...
episode: 1000 loss tensor(0.9569, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33299629255139873
gzsl: unseen=0.1577 , seen=0.1939 , h=0.1739
____________________________________________________________________________________________________
episode: 2000 loss tensor(0.7471, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3923154701718908
gzsl: unseen=0.1635 , seen=0.2846 , h=0.2077
____________________________________________________________________________________________________
episode: 3000 loss tensor(0.6647, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.37209302325581395
gzsl: unseen=0.1810 , seen=0.2727 , h=0.2176
____________________________________________________________________________________________________
episode: 4000 loss tensor(0.3560, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3734411863835524
gzsl: unseen=0.1550 , seen=0

episode: 31000 loss tensor(0.3224, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.32490731378496795
gzsl: unseen=0.2039 , seen=0.2290 , h=0.2157
____________________________________________________________________________________________________
episode: 32000 loss tensor(0.3525, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3333333333333333
gzsl: unseen=0.1985 , seen=0.2262 , h=0.2115
____________________________________________________________________________________________________
episode: 33000 loss tensor(0.5385, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34984833164812945
gzsl: unseen=0.2133 , seen=0.2234 , h=0.2182
____________________________________________________________________________________________________
episode: 34000 loss tensor(0.3721, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.34041118975396023
gzsl: unseen=0.2235 , seen=0.2319 ,

episode: 61000 loss tensor(0.1748, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.32221098752949107
gzsl: unseen=0.1951 , seen=0.2358 , h=0.2136
____________________________________________________________________________________________________
episode: 62000 loss tensor(0.4537, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3282777216043141
gzsl: unseen=0.1726 , seen=0.2415 , h=0.2013
____________________________________________________________________________________________________
episode: 63000 loss tensor(0.3238, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.32221098752949107
gzsl: unseen=0.2096 , seen=0.2183 , h=0.2139
____________________________________________________________________________________________________
episode: 64000 loss tensor(0.3998, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3185035389282103
gzsl: unseen=0.2019 , seen=0.2228 , 

episode: 91000 loss tensor(0.3519, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.33131108864172565
gzsl: unseen=0.1999 , seen=0.2285 , h=0.2132
____________________________________________________________________________________________________
episode: 92000 loss tensor(0.2296, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3120997640714526
gzsl: unseen=0.1759 , seen=0.2251 , h=0.1975
____________________________________________________________________________________________________
episode: 93000 loss tensor(0.2315, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3282777216043141
gzsl: unseen=0.1874 , seen=0.2364 , h=0.2091
____________________________________________________________________________________________________
episode: 94000 loss tensor(0.5546, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3228850690933603
gzsl: unseen=0.2042 , seen=0.2273 , h

zsl: 0.33602965958881026
gzsl: unseen=0.2356 , seen=0.2177 , h=0.2263
____________________________________________________________________________________________________
episode: 121000 loss tensor(0.3470, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3282777216043141
gzsl: unseen=0.1985 , seen=0.2387 , h=0.2167
____________________________________________________________________________________________________
episode: 122000 loss tensor(0.5325, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.32726659925851026
gzsl: unseen=0.2356 , seen=0.2132 , h=0.2238
____________________________________________________________________________________________________
episode: 123000 loss tensor(0.1819, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3356926188068756
gzsl: unseen=0.2336 , seen=0.2137 , h=0.2232
_______________________________________________________________________________________________

...

episode: 180000 loss tensor(0.2371, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.31681833501853723
gzsl: unseen=0.2238 , seen=0.2058 , h=0.2144
____________________________________________________________________________________________________
episode: 181000 loss tensor(0.4198, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.31580721267273343
gzsl: unseen=0.2214 , seen=0.2137 , h=0.2175
____________________________________________________________________________________________________
episode: 182000 loss tensor(0.2303, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3134479271991911
gzsl: unseen=0.2245 , seen=0.2069 , h=0.2153
____________________________________________________________________________________________________
episode: 183000 loss tensor(0.2316, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.32490731378496795
gzsl: unseen=0.2302 , seen=0.21

episode: 210000 loss tensor(0.1271, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3127738456353219
gzsl: unseen=0.2201 , seen=0.2058 , h=0.2127
____________________________________________________________________________________________________
episode: 211000 loss tensor(0.6068, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3087293562521065
gzsl: unseen=0.2144 , seen=0.2080 , h=0.2112
____________________________________________________________________________________________________
episode: 212000 loss tensor(0.3523, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3181664981462757
gzsl: unseen=0.2268 , seen=0.2098 , h=0.2180
____________________________________________________________________________________________________
episode: 213000 loss tensor(0.3673, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.31648129423660265
gzsl: unseen=0.2275 , seen=0.2109

episode: 240000 loss tensor(0.3231, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3134479271991911
gzsl: unseen=0.2302 , seen=0.2058 , h=0.2173
____________________________________________________________________________________________________
episode: 241000 loss tensor(0.5493, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3154701718907988
gzsl: unseen=0.2265 , seen=0.2103 , h=0.2181
____________________________________________________________________________________________________
episode: 242000 loss tensor(0.1385, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.31681833501853723
gzsl: unseen=0.2231 , seen=0.2177 , h=0.2204
____________________________________________________________________________________________________
episode: 243000 loss tensor(0.2597, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.316144253454668
gzsl: unseen=0.2299 , seen=0.2092 

episode: 270000 loss tensor(0.5443, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.320525783619818
gzsl: unseen=0.2123 , seen=0.2177 , h=0.2150
____________________________________________________________________________________________________
episode: 271000 loss tensor(0.5460, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.32659251769464104
gzsl: unseen=0.2346 , seen=0.2092 , h=0.2212
____________________________________________________________________________________________________
episode: 272000 loss tensor(0.4303, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3195146612740142
gzsl: unseen=0.2282 , seen=0.2132 , h=0.2204
____________________________________________________________________________________________________
episode: 273000 loss tensor(0.4619, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.316144253454668
gzsl: unseen=0.2305 , seen=0.2098 ,

episode: 300000 loss tensor(0.2792, device='cuda:0')
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3185035389282103
gzsl: unseen=0.2238 , seen=0.2149 , h=0.2192
____________________________________________________________________________________________________
